# Modelo de recomendación

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Cargamos datasets

In [2]:
steam_games = pd.read_csv('datasets/steam_games.csv')

Tratamiento para optimizar el trabajo

In [3]:
# Seleccionar columnas y limpiar 'specs'
df = steam_games.loc[:, ["specs", "item_id", "app_name"]].assign(
    specs=lambda x: x['specs'].astype(str).str.replace(r"[\[\]']", "", regex=True))

# Convertir 'item_id' a tipo entero
df["item_id"] = df["item_id"].astype(int)


In [4]:
# Especificar parámetros
cv = CountVectorizer(lowercase=True, stop_words='english')

# Obtener la matriz dispersa
vectores_sparse = cv.fit_transform(df['specs'])

# Si deseas una matriz densa:
vectores_dense = vectores_sparse.toarray()

In [5]:
# Calcular la similitud del coseno entre vectores
similitud = cosine_similarity(vectores_dense)

In [6]:
def recomendacion(juego, df, similitud, juego_a_indice):
    indice_juego = juego_a_indice.get(juego)
    if indice_juego is None:
        return "Juego no encontrado en la base de datos."

    distances = similitud[indice_juego]
    similar_indices = distances.argsort()[::-1][1:6]  # Índices de los juegos más similares, excluyendo el propio juego
    recommended_titles = df.iloc[similar_indices]['app_name'].tolist()
    
    return recommended_titles

# Precomputar el mapeo de títulos de juegos a índices
juego_a_indice = {juego: indice for indice, juego in enumerate(df['item_id'])}

# Ejemplo de uso:
recomendaciones = recomendacion("Nombre del juego", df, similitud, juego_a_indice)

# Aplicar la función a la columna 'item_id' y crear una nueva columna 'Recomendaciones'
df['Recomendaciones'] = df['item_id'].apply(recomendacion, df=df, similitud=similitud, juego_a_indice=juego_a_indice)


In [9]:
# df.drop(columns=['app_name', 'specs'], inplace=True)
df

,item_id,Recomendaciones
0,761140,"[The Valley In My Mind, The Great Jitters: Pud..."
1,643980,"[Ironbound, Warhammer 40,000: Regicide, KROSMA..."
2,670290,"[Hunger Dungeon, Lords of War, Heroes of Parag..."
3,767400,"[The Valley In My Mind, The Great Jitters: Pud..."
4,772540,"[AL･FINE, Only If, Defense Warfare, Air Dash, ..."
...,...,...
27176,745400,"[Lost Grimoires 3: The Forgotten Well, Nebula ..."
27177,773640,"[AL･FINE, Only If, Defense Warfare, Air Dash, ..."
27178,733530,"[Crushing Blow, Quantz, Sentinels, Flight Cont..."
27179,610660,"[Puzzle Box, The Men of Yoshiwara: Ohgiya, Pap..."


In [10]:
df.to_csv('datasets/model_recommend.csv')